In [ ]:
import pandas as pd, random
# %pip install requests
# %pip install lxml
import requests
from io import StringIO

url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
html_content = response.text
html_string = StringIO(html_content)
df_list = pd.read_html(html_string)


for df in df_list:
    print(df.head())

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   --------------- ------------------------ 1.6/4.0 MB 8.4 MB/s eta 0:00:01
   --------------------------------- ------ 3.4/4.0 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
    0                                                  1
0 NaN  This section needs additional citations for ve...
  Rank             Player    Club                 Country and other    Total  \
  Rank             Player  League Cup Continental Country and other    Total   
0    1  Cristiano Ronaldo  576[a]  57         172               141  946[65]   
1    2      Lionel Messi*  542[b]  71         157               114  884[68]   
2    3              Pelé*  604[c]  49          26                83      762   
3    4            Romário  545[d]  93          54                64      756   
4    5      Ferenc Puskás  516[e]  69      

In [3]:
df = df_list[3]
df

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989+,582,1.70,1924–1951
1,2,Cristiano Ronaldo,956,1317,0.73,2002–present
2,3,Josef Bican,950+,624,1.52,1930–1957
3,4,Ronnie Rooke,934+,1030,0.91,1929–1961
4,5,Lionel Messi,901,1168,0.77,2003–present
...,...,...,...,...,...,...
77,78,Steve Bloomer,504,755,0.67,1891–1920
78,79,Albert de Cleyn,503,588,0.86,1933–1955
79,80,Arthur Rowley,502+,729,0.69,1941–1967
80,81,Delio Onnis,502,747,0.67,1966–1986


In [ ]:
# career starte date objects
random.seed(22)
df['career'] = df['Career span'].apply(lambda x: f'{x.split("–")[0]}-{random.choice(range(1,13)):02d}-{random.choice(range(1,29)):02d}')
df['career'].head() 

0    1924-03-08
1    2002-01-20
2    1930-08-06
3    1929-12-04
4    2003-12-21
Name: career, dtype: object

In [11]:
df['career'] = pd.to_datetime(df['career'])
df['career'].head()

0   1924-03-08
1   2002-01-20
2   1930-08-06
3   1929-12-04
4   2003-12-21
Name: career, dtype: datetime64[ns]

In [16]:
# play with datetimes
df['career'].dt.day_of_week

0     5
1     6
2     2
3     2
4     6
     ..
77    4
78    2
79    0
80    1
81    0
Name: career, Length: 82, dtype: int32

In [17]:
df['Goals'] = df['Goals'].apply(lambda x: int(x.replace("+","")))
df['Matches'] = df['Matches'].apply(lambda x: int(x.replace("+", "")))
df.head()


,Rank,Player,Goals,Matches,Ratio,Career span,career
0,1,Erwin Helmchen,989,582,1.70,1924–1951,1924-03-08
1,2,Cristiano Ronaldo,956,1317,0.73,2002–present,2002-01-20
2,3,Josef Bican,950,624,1.52,1930–1957,1930-08-06
3,4,Ronnie Rooke,934,1030,0.91,1929–1961,1929-12-04
4,5,Lionel Messi,901,1168,0.77,2003–present,2003-12-21


In [24]:
from datetime import timedelta
data = []
columns = ["player", "goals", "game_date", "shots", 
           "outcome"]
for row in df.itertuples():
    name = row.Player
    matches = row.Matches
    mean = row.Ratio
    sigma = random.choice([0.5, 1.0, 1.5, 2])
    start = row.career
    for i in range(matches):
        player = name
        goals = round(random.normalvariate(mean, sigma))
        goals = goals if goals > 0 else 0
        shots = goals + random.choice([0,1,2,3,4])
        game_date = start + timedelta(days=i * 4)
        outcome = random.choice(["win", "loss", "tie"])
        g_data = [player, goals, game_date, shots, outcome]
        data.append(g_data)

games_df = pd.DataFrame(data=data, columns=columns)
games_df     
        
    

,player,goals,game_date,shots,outcome
0,Erwin Helmchen,1,1924-03-08,2,loss
1,Erwin Helmchen,2,1924-03-12,6,win
2,Erwin Helmchen,1,1924-03-16,3,tie
3,Erwin Helmchen,1,1924-03-20,5,loss
4,Erwin Helmchen,2,1924-03-24,3,win
...,...,...,...,...,...
59251,Lajos Tichy,0,1958-12-08,2,tie
59252,Lajos Tichy,0,1958-12-12,2,tie
59253,Lajos Tichy,0,1958-12-16,0,loss
59254,Lajos Tichy,2,1958-12-20,4,tie


In [31]:
# agg_df = games_df.groupby(by="player").agg(
#     {
#         "goals": "sum",
#         "shots": "sum",
#         "outcome": "count"
#     }
# ).reset_index()
agg_df = games_df.groupby(by="player").agg(
    goals=("goals", "sum"),
    shots=("shots", "sum"),
    matches=("player", "count"),
).reset_index()
agg_df.sort_values(by="goals", ascending=False)

,player,goals,shots,matches
56,Lionel Messi,1203,3498,1168
6,Cristiano Ronaldo,1173,3838,1317
18,Ferenc Bene,1158,3073,972
20,Ferenc Puskás,1136,2739,792
42,Jimmy Jones,1130,2614,760
...,...,...,...,...
26,George Brown,529,1948,711
64,Roberto Dinamite,524,2285,869
8,David McLean,521,2049,801
33,György Sárosi,508,1725,607


In [32]:
df1 = pd.read_csv("top500footballers.csv")
df1

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989+,582,1.70,1924–1951
1,2,Josef Bican,950+,624,1.52,1930–1957
2,3,Ronnie Rooke,934+,1030,0.91,1929–1961
3,4,Cristiano Ronaldo,934,1291,0.72,2002–present
4,5,Lionel Messi,879,1140,0.77,2003–present
...,...,...,...,...,...,...
74,75,Albert de Cleyn,503,588,0.86,1933–1955
75,76,George Camsell,503+,590,0.85,1923–1942
76,77,Arthur Rowley,502+,729,0.69,1941–1967
77,78,Delio Onnis,502,747,0.67,1966–1986
